In [ ]:
from qiskit_aer import AerSimulator

from qlbm.components import EmptyPrimitive
from qlbm.components.lqlga.initial import LQGLAInitialConditions
from qlbm.components.lqlga.lqlga import LQLGA
from qlbm.components.lqlga.measurement import LQLGAGridVelocityMeasurement
from qlbm.infra import QiskitRunner, SimulationConfig
from qlbm.lattice.lattices.lqlga_lattice import LQLGALattice
from qlbm.tools.utils import create_directory_and_parents


In [ ]:
from qlbm.components.lqlga import LQGLAInitialConditions
from qlbm.lattice import LQLGALattice

lattice = LQLGALattice(
    {
        "lattice": {
            "dim": {"x": 7},
            "velocities": "D1Q3",
        },
        "geometry": [{"shape": "cuboid", "x": [3, 5], "boundary": "bounceback"}],
    },
)


output_dir = "qlbm-output/lqlga-d1q3-7-qiskit"
create_directory_and_parents(output_dir)


In [ ]:
cfg = SimulationConfig(
    initial_conditions=LQGLAInitialConditions(
        lattice, [(tuple([2]), (True, True, True))]
    ),
    algorithm=LQLGA(lattice),
    postprocessing=EmptyPrimitive(lattice),
    measurement=LQLGAGridVelocityMeasurement(lattice),
    target_platform="QISKIT",
    compiler_platform="QISKIT",
    optimization_level=0,
    statevector_sampling=True,
    execution_backend=AerSimulator(method="statevector"),
    sampling_backend=AerSimulator(method="statevector"),
)

In [ ]:
cfg.prepare_for_simulation()

# Number of shots to simulate for each timestep when running the circuit
NUM_SHOTS = 2**10

# Number of timesteps to simulate
NUM_STEPS = 20

# Create a runner object to simulate the circuit
runner = QiskitRunner(
    cfg,
    lattice,
)

# Simulate the circuits using both snapshots
runner.run(
    NUM_STEPS,  # Number of time steps
    NUM_SHOTS,  # Number of shots per time step
    output_dir,
    statevector_snapshots=True,
)